In [78]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import preprocessing
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from IPython.display import clear_output

In [79]:
VOCAB_SIZE = 500000
(train_data,train_labels),(test_data,test_labels) = imdb.load_data(num_words=VOCAB_SIZE)
print(train_data[0])
clear_output()

In [80]:
train_data = preprocessing.sequence.pad_sequences(train_data,250)
test_data = preprocessing.sequence.pad_sequences(test_data,250)


In [81]:
vocabulary = imdb.get_word_index()
def encode_str(text:str):
    encoded = preprocessing.text.text_to_word_sequence(text)
    data_ = [vocabulary[word] if word in vocabulary else 0 for word in encoded]
    return preprocessing.sequence.pad_sequences([data_],250)[0]
# print(encode_str("I am a Fool"))

def reverse_dict(dict_original):
    return {value: key for (key, value) in dict_original.items()}

def decode_str(encoded):
    vocal_rev = reverse_dict(vocabulary)
    txt = ''
    for num in encoded:
        if num != 0:
            txt += vocal_rev[num] + " "
    return txt
en = encode_str("As I am Here But I am There")
# print(en)
# print(decode_str(en))

In [82]:
model = keras.models.Sequential([
    keras.layers.Embedding(VOCAB_SIZE,40),
    keras.layers.LSTM(40),
    keras.layers.Dense(1,activation='sigmoid')
])

In [83]:
model.compile(loss=keras.losses.BinaryCrossentropy(),optimizer='adam',metrics=['acc'])

In [84]:
class CallBack(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('loss') >= 0.14:
            print("Loss Equal To .18 Ending Training ")
            self.model.stop_training = True
c = CallBack()
his = model.fit(train_data,train_labels,epochs=2,callbacks=[c])
model.save('Movie',overwrite=True)

Epoch 1/2
782/782 [==============================] - 209s 267ms/step - loss: 0.3896 - acc: 0.8185
INFO:tensorflow:Assets written to: Movie/assets


In [85]:
loss,acc = model.evaluate(test_data,test_labels)
print(acc)

782/782 [==============================] - 9s 12ms/step - loss: 0.3456 - acc: 0.8498

In [94]:
def make_prediction(text:str):
    encoded = encode_str(text)
    pred = model.predict(np.array([encoded]))
    return pred
positive_review = "That movie was! really loved it and would great watch it again because it was amazingly great"
negative_review = "that movie really sucked. I hated it and wouldn't watch it again. Was one of the worst things I've ever watched"
res = make_prediction(positive_review)

if res > 0.5:
    print("Positive Review")
else:
    print("Negative Review")


Positive Review
